In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset_dir = "/root/skripsi/datasets/"
holdout_dir = "holdout/"

In [3]:
df_t = pd.read_csv(dataset_dir + "liver/liver.csv", header=None)

In [4]:
display(df_t.head())

,0,1,2,3,4,5,6,7,8,9,10
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [5]:
display(df_t.isnull().any())

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9      True
10    False
dtype: bool

In [6]:
display(df_t[df_t.isnull().any(axis=1)])

,0,1,2,3,4,5,6,7,8,9,10
209,45,Female,0.9,0.3,189,23,33,6.6,3.9,NaN,1
241,51,Male,0.8,0.2,230,24,46,6.5,3.1,NaN,1
253,35,Female,0.6,0.2,180,12,15,5.2,2.7,NaN,2
312,27,Male,1.3,0.6,106,25,54,8.5,4.8,NaN,2


In [7]:
display(df_t.shape)

(583, 11)

In [8]:
df = df_t.dropna()
display(df.shape)

(579, 11)

In [9]:
df_np = df.values
X = df_np[:, :-1]
y = np.array([1 if u == 2 else 0 for u in df_np[:, -1]])

In [10]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

enc.fit(X[:, [1]])
display(enc.categories_)

[array(['Female', 'Male'], dtype=object)]

In [11]:
X[:, [1]] = enc.transform(X[:, [1]]).astype(np.int)

In [12]:
display(X[:5])
display(y[:5])

array([[65, 0, 0.7, 0.1, 187, 16, 18, 6.8, 3.3, 0.9],
       [62, 1, 10.9, 5.5, 699, 64, 100, 7.5, 3.2, 0.74],
       [62, 1, 7.3, 4.1, 490, 60, 68, 7.0, 3.3, 0.89],
       [58, 1, 1.0, 0.4, 182, 14, 20, 6.8, 3.4, 1.0],
       [72, 1, 3.9, 2.0, 195, 27, 59, 7.3, 2.4, 0.4]], dtype=object)

array([0, 0, 0, 0, 0])

In [13]:
n_holdout = 10

In [14]:
from sklearn.model_selection import StratifiedKFold

full_df = pd.DataFrame(np.hstack([X, np.expand_dims(y, axis=-1)]))
full_df.to_csv(holdout_dir + "liver.csv", index=False)

holdout_skf = StratifiedKFold(n_splits=n_holdout, shuffle=True, random_state=42)
for i, (train_id, test_id) in enumerate(holdout_skf.split(X, y)):
    X_train = X[train_id]
    y_train = y[train_id]
    X_test = X[test_id]
    y_test = y[test_id]
    
    data_train = np.hstack([X_train, np.expand_dims(y_train, axis=-1)])
    data_test = np.hstack([X_test, np.expand_dims(y_test, axis=-1)])
    
    df_train = pd.DataFrame(data_train)
    df_test = pd.DataFrame(data_test)

    df_train.to_csv(holdout_dir + ("liver-%d-train.csv" % i), index=False)
    df_test.to_csv(holdout_dir + ("liver-%d-test.csv" % i), index=False)